# Step 2: Private Prediction using Syft Keras - Serving (Client)

_Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials_

Congratulations! After training your model with normal Keras and securing it with Syft Keras, you are ready to request some private predictions. 

In [3]:
pip install syft

     |████████████████████████████████| 286kB 2.8MB/s 
     |████████████████████████████████| 256kB 49.8MB/s 
     |████████████████████████████████| 204kB 51.7MB/s 
     |████████████████████████████████| 2.1MB 52.1MB/s 
     |████████████████████████████████| 81kB 27.8MB/s 
     |████████████████████████████████| 460kB 46.4MB/s 
     |████████████████████████████████| 389kB 49.0MB/s 
     |████████████████████████████████| 51kB 17.9MB/s 
     |████████████████████████████████| 266kB 52.1MB/s 
     |████████████████████████████████| 122kB 47.7MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067082 sha256=66952fde45ad8837c90705485f9b7f543e5c318a9f53abb3f24e4d3deb712f74
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=95d0bcf528fcdc0461223ca0c5fd16c7a16eec1cdd54ddaa75110fbf6d58288a
  Stored in di

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import syft as sy

W0826 16:46:49.061327 139877618374528 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0826 16:46:49.078113 139877618374528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [5]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
num_classes = 10
input_shape = (1, 28, 28, 1)
output_shape = (1, num_classes)

In [7]:
client = sy.TFEWorker()

alice = sy.TFEWorker(host='localhost:4000')
bob = sy.TFEWorker(host='localhost:4001')
carol = sy.TFEWorker(host='localhost:4002')
cluster = sy.TFECluster(alice, bob, carol)

client.connect_to_model(input_shape, output_shape, cluster)

W0826 16:48:02.842378 139877618374528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/tensor/native.py:403: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 16:48:02.857806 139877618374528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/config.py:300: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0826 16:48:02.860337 139877618374528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/config.py:87: The name tf.GraphOptions is deprecated. Please use tf.compat.v1.GraphOptions instead.

I0826 16:48:02.861839 139877618374528 session.py:55] Starting session on target 'grpc://localhost:4000' using config graph_options {
}



In [0]:
# User inputs
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [0]:
for image, expected_label in zip(images, expected_labels):

    res = client.query_model(image.reshape(1, 28, 28, 1))
    predicted_label = np.argmax(res)

    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "wrongly",
        predicted_label))